In [1]:
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
train = pd.read_csv(r'C:\Users\ALi\OneDrive - MMC\Desktop\Kaggle\Ins\train.csv')
train.describe

<bound method NDFrame.describe of               id   Age  Gender  Annual Income Marital Status  \
0              0  19.0  Female        10049.0        Married   
1              1  39.0  Female        31678.0       Divorced   
2              2  23.0    Male        25602.0       Divorced   
3              3  21.0    Male       141855.0        Married   
4              4  21.0    Male        39651.0         Single   
...          ...   ...     ...            ...            ...   
1199995  1199995  36.0  Female        27316.0        Married   
1199996  1199996  54.0    Male        35786.0       Divorced   
1199997  1199997  19.0    Male        51884.0       Divorced   
1199998  1199998  55.0    Male            NaN         Single   
1199999  1199999  21.0  Female            NaN       Divorced   

         Number of Dependents Education Level     Occupation  Health Score  \
0                         1.0      Bachelor's  Self-Employed     22.598761   
1                         3.0        Mast

In [3]:
column_count = len(train.columns)
print(train.columns)
print("There are this many columns: " + str(column_count))

Index(['id', 'Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount'],
      dtype='object')
There are this many columns: 21


In [4]:
train.head(10)

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
5,5,29.0,Male,45963.0,Married,1.0,Bachelor's,NaN,33.053198,Urban,...,2.0,4.0,614.0,5.0,2022-05-20 15:21:39.207847,Average,No,Weekly,House,3202.0
6,6,41.0,Male,40336.0,Married,0.0,PhD,NaN,NaN,Rural,...,2.0,8.0,807.0,6.0,2020-02-21 15:21:39.219432,Poor,No,Weekly,House,439.0
7,7,48.0,Female,127237.0,Divorced,2.0,High School,Employed,5.769783,Suburban,...,1.0,11.0,398.0,5.0,2022-08-08 15:21:39.181605,Average,No,Rarely,Condo,111.0
8,8,21.0,Male,1733.0,Divorced,3.0,Bachelor's,NaN,17.869551,Urban,...,1.0,10.0,685.0,8.0,2020-12-14 15:21:39.198406,Average,No,Monthly,Condo,213.0
9,9,44.0,Male,52447.0,Married,2.0,Master's,Employed,20.473718,Urban,...,1.0,9.0,635.0,3.0,2020-08-02 15:21:39.144722,Poor,No,Daily,Condo,64.0


In [5]:
from sklearn.model_selection import train_test_split
# Separate the independent and dependent variables from the training set
X, y = train.drop(["id","Premium Amount"], axis=1), train[["Premium Amount"]]

In [6]:
# Separate the columns with categorical/text values
cats = X.select_dtypes(exclude=np.number).columns.to_list()

# Convert all object types to category types
for col in cats:
    X[col] = X[col].astype('category')

In [7]:
X.dtypes

Age                      float64
Gender                  category
Annual Income            float64
Marital Status          category
Number of Dependents     float64
Education Level         category
Occupation              category
Health Score             float64
Location                category
Policy Type             category
Previous Claims          float64
Vehicle Age              float64
Credit Score             float64
Insurance Duration       float64
Policy Start Date       category
Customer Feedback       category
Smoking Status          category
Exercise Frequency      category
Property Type           category
dtype: object

In [8]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [9]:
# create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)

# Note: replace with the test set? remove the training/test split?
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [10]:
# Define hyperparameters. Use 'gpu_hist' if you are using NVIDIA GPU, but use 'hist' if just using CPU
params = {"objective": "reg:squarederror", "tree_method": "hist"}

n = 100
model = xgb.train(
    params=params,
    dtrain = dtrain_reg,
    num_boost_round=n
)

In [13]:
# Evaluation
from sklearn.metrics import mean_absolute_error

# Looks like the test set doesn't have the 'Premium Amount' variable
X_test = pd.read_csv(r'C:\Users\ALi\OneDrive - MMC\Desktop\Kaggle\Ins\test.csv')
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

preds = model.predict(dtest_reg)
rmse = mean_absolute_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse: .3f}")


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Gender: object, Marital Status: object, Education Level: object, Occupation: object, Location: object, Policy Type: object, Policy Start Date: object, Customer Feedback: object, Smoking Status: object, Exercise Frequency: object, Property Type: object